<a href="https://colab.research.google.com/github/pawntoqueen/NLP-examples/blob/main/hepsiburada_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# Insert the directory
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [4]:
import pandas as pd
ds_size = 10000

In [5]:
INPUT_FOLDER = '/content/drive/My Drive/Colab Notebooks/hepsiburada_reviews_dataset/product_reviews.txt'
data = pd.read_csv(INPUT_FOLDER, sep=";", names = ['review', 'label'])
data = data.sample(frac = 1).reset_index(drop = True)

In [6]:
data.head()

,review,label
0,kaliteli bi şey beklemeyin kesinlikle ses çok ...,neg
1,mesleğimiz gereği oldukça fazla sayıda a4 kağı...,neg
2,oyuncak niyeti ile tek kullanımlık alırsanız a...,neg
3,bu markanın köpek mamasını da aldım. o ürünün ...,neg
4,i?yi ürün güzel fiyat hızlı kargo daha ne olsu...,pos


In [7]:
import string
import re
import nltk
from nltk.corpus import stopwords as sw
nltk.download('stopwords')
punc = string.punctuation
useless = sw.words('turkish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
print(punc)
print(useless)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [9]:
!pip install zemberek-python


     |████████████████████████████████| 93.6 MB 13 kB/s 
     |████████████████████████████████| 117 kB 67.2 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.2-py3-none-any.whl size=144567 sha256=721278d18b997c100e9abda6f49c1249cc6680a8e73b184852ef46939373d2ae
  Stored in directory: /root/.cache/pip/wheels/14/4c/18/1dbbc9875a2547d2063400ea9f404da4af3331965a71061029
Successfully built antlr4-python3-runtime


In [10]:

from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)


2021-08-24 10:46:04,566 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 9.010739088058472



In [11]:
#her kelimeyi küçük harfe dönüştürmek
clone_string_data_frame = data['review'].apply(lambda x:" ".join(x.lower() for x in str(x).split()))
print(type(clone_string_data_frame))

#zemberek normalizasyon işlemi
clone_string_data_frame = clone_string_data_frame.apply(lambda x: " ".join(normalizer.normalize(x) for x in str(x).split()) )

#noktalama işaretlerini silmek
clone_string_data_frame = clone_string_data_frame.str.replace("[^\w\s]", "")

#sayıları silmek
clone_string_data_frame = clone_string_data_frame.str.replace("\d", "")

# türkçe stopwords'leri silmek
clone_string_data_frame = clone_string_data_frame.apply(lambda x : " ".join(x for x in str(x).split() if x not in useless ))

#Seriyi DataFrame'e dönüştürmek
df = pd.DataFrame(clone_string_data_frame, columns = ['review', 'label'])
df['label'] = data['label']

<class 'pandas.core.series.Series'>


In [12]:
df.to_csv(r'./cleaned.csv', index=False)

In [13]:
data = pd.read_csv('cleaned.csv', sep=",")
data.head(ds_size)

,review,label
0,kaliteli bir beklemeyin kesinlikle ses yapıyor,neg
1,mesleğimiz gereği oldukça fazla sayıda a kağıd...,neg
2,oyuncak niyeti tek kullanımlık alırsanız amenn...,neg
3,markanın köpek mamasını aldım ürünün altına be...,neg
4,ı yi ürün güzel fiyat hızlı kargo olsun teşekk...,pos
...,...,...
9995,ı işlevsel fakat ay kullandıktan sonra kapıya ...,neg
9996,güzel dönüyor adeta taş kesiliyor yapıştırdığı...,neg
9997,ürün mükemmel,pos
9998,ürün gün içinde teslim oldu vestel servisi ara...,pos


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data['review'].head(ds_size).values.astype('U'),
                                                    data['label'].head(ds_size).values.astype('U'), test_size = 0.1, random_state =42)

print( X_train.shape)
print(X_test.shape)
print( Y_train.shape)
print(Y_test.shape)

(9000,)
(1000,)
(9000,)
(1000,)


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect =  CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

(9000, 23659)


In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)


(9000, 23659)


In [17]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, Y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [18]:
y_pred = clf.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:], y_pred[:]):
  print('%r => %s' % (review, sentiment))

'ben ürünü yaylada kullandım gayet kullanışlı bilene göre tabii yanlız dönüşleri biraz zor birde cıvataları kadar sıkıysamda tabladakiler düştü diğer cıvatalar duruyor' => neg
'fiyata ürün başka bir yerde bulmak imkansız kargo hızlı' => pos
'koku yapıyor işitmiyor ucuz almaya gerek yok ben iade ettim' => neg
'uçurtma avcısı bin muhteşem güneş isimli kitapları severek okumuştum kitabında onların kadar güzel olacağını düşünmüştüm ancak yanılmışım tavsiye etmiyorum' => neg
'nnt bor kimlik olan ürünü kullanma talimatlarına göre uyguladım diğer arkadaşların yazdığı mükemmel değişimlerin hiçbiri benim aracımda olmadı ürünü almamda yorumların etkisi ölmüştü arada aracım fiesta tdçi km aracımın performansından memnundum zaten bir değişiklikte olmadı motorun sesini olsa kesmesini umuyordum büyük değişimler bekleyerek ürünü almayın' => neg
'lik yazan ürün yapraktan oluşmakta tek tek yerleştirirseniz kartvizit almaktadır' => neg
'normal bir ci haz fi yata alınır dersen alınır' => pos
'bence fiyat

In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,y_pred))

0.876
